# Roulette Strategy

A friend sent me this [video](https://www.instagram.com/p/C4_6h2yxLvL/), showing a proposed strategy for placing bets in roulette. 
Then he asked "What is the probability of making money doing this 10 times?"

Interesting question! Let's start off with the basics of roulette.

## How Roulette Works

We can find everything we need on the Wikipedia page here: https://en.wikipedia.org/wiki/Roulette

But here's a summary of what's relevant to the video and our calculations: 

### The Layout

A spin of the roulette wheel results in a single number, which will also have a color associated with it (red, black, or green).

In an American-style layout, as is shown in the video, there are numbers 1-36, as well as 0 and 00, for a total of **38 equally likely outcomes** on a single spin.

### Betting

There are many different varieties of bets, but they all effectively boil down to choosing a set of numbers as possible outcomes and you win if the outcome is in your set.

#### Payout

The payout is given by

$$ V = \frac{36}{n} $$ 

Where:
- $V$ is the ratio to the initial bet received upon success. Note that this includes the initial bet: subtract 1 to get the profit margin.
- $n$ is the size of the set of numbers you bet on.

The types of bets which are allowed have sizes $n \in \{1, 2, 3, 4, 6, 12, 18\}$, so we always have whole-number payouts. The only exception is the "American-style top line bet", which is a bet on $\{0, 00, 1, 2, 3\}$, which has $n=5$ and therefore should payout $V=\frac{36}{5} = 7.2$ times the initial bet, but is rounded down to $7$ to keep whole numbers: i.e. it pays out less than it should.

#### Probability

The probability of any of the number in a set of size $n$ being chosen is just

$$ P = \frac{n}{38} $$ 

Where:
- $P$ is the probability of success of the bet.
- $n$ is the size of the set of numbers you bet on.

#### Expected Value (Utility)

Putting these together the expected return of any bet is

$$ U = PV = \frac{n}{38} \frac{36}{n} = \frac{36}{38} \approx 0.9474 $$

Where
- $U$ is the expected value of the bet (utility).
- $P$ is the probability of success of the bet.
- $V$ is the ratio to the initial bet received upon success. Note that this includes the initial bet: subtract 1 to get the profit margin.
- $n$ is the size of the set of numbers you bet on.

So therefore every bet you can makes has the same expected value, which is less than your initial bet. **Effectively you can expect to lose $0.0526$ of your initial bet every round.**

## The Proposed Strategy

In the video, it is claimed that there is a roulette strategy which will win 70% of the time. 
It goes as follows:

- The board is American-style
- \\$25 on the first dozen, $\{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12\}$
- \\$25 on the second dozen, $\{13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24\}$
- \\$25 on the first column, $\{1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34\}$
- \\$25 on the third column, $\{3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36\}$

### Expected Value

While there is overlap between the numbers chosen for the parts of the bet, each part effectively acts as an independent bet and so the expected value of the bet is unchanged: \\$94.74, so a loss of \\$5.26.

### Breakdown

Let's go through the possible outcomes with the overlaps. Note that since each part of the bet is on a set of 12 numbers, the returns will always be 3 times, which makes the math easy.
- \\$150 for an overlap, $|\{1, 3, 4, 6, 7, 9, 10, 12, 13, 15, 16, 18, 19, 21, 22, 24\}| = 16$
- \\$75 for a single, $|\{2, 5, 8, 11, 14, 17, 20, 23, 25, 27, 28, 30, 31, 33, 34, 36\}| = 16$
- \\$0 for a miss, $|\{00, 0, 26, 29, 32, 35\}| = 6$

We can again compute the expected return to confirm:

$$ U = \$150 \frac{16}{38} + \$75 \frac{16}{38} + \$0 \frac{6}{38} \approx \$94.74 $$

Only 1 of the above cases leads to making money on the bet, with a probability of $ \frac{16}{38} \approx 0.4211 $.

The probability of getting back any chips is $ \frac{32}{38} \approx 0.8421 $.

None of these values are 70%, so where did this number come from? It seems this is just a flat out lie.

## Answering The Question

Okay, but now we still need to answer the question, "What is the probability of making money doing this 10 times?"

### The Easier Question: Expected Value Again

We can start off by more easily answering the question of what is the expected value of doing this 10 times: since each bet has an expected loss of \\$5.26, 10 times is an expected loss of \\$52.63.


### Computation via State Space

An easy way to compute the answer is to keep track of how the probability distribution through the state space evolves after 10 steps.

The state is simply how much money we've gained/lost. Since each part of the bet is in \\$25, we could divide through by the factor of \\$25 to see by which factor we're up or down. In which case the possible results become:

- $\frac{16}{38}$ probability of $+2$
- $\frac{16}{38}$ probability of $-1$
- $\frac{6}{38}$ probability of $-4$

So the maximum state is $2 \times 10 = 20$ and the minimum state is $-4 \times 10 = -40$, for a maximum total number of states of $20 - (-40) = 60$, computed over 10 steps is 600 operations. Easy to compute.

Here we'll do it in Python, and use a dictionary to keep track of the probability of being in each state, so we can keep our states in terms of dollars from the original betting amounts at no extra overhead.

In [1]:
from collections import defaultdict

In [2]:
# initial state
P = {0:1}

# Perform the steps
for step in range(10):
    Pnew = defaultdict(float)
    for state,p in P.items():
        Pnew[state+50] += p * 16/38
        Pnew[state-25] += p * 16/38
        Pnew[state-100] += p * 6/38
    P = Pnew

# Sum all final states where we make money
result = sum(p for state,p in P.items() if state > 0)

In [3]:
P

defaultdict(float,
            {500: 0.0001751313358641004,
             425: 0.001751313358641004,
             350: 0.008537652623374893,
             275: 0.02692644288910543,
             200: 0.06152856385787964,
             125: 0.10816549131306498,
             50: 0.15166647328543378,
             -25: 0.1735852545396751,
             -100: 0.1646255935459441,
             -175: 0.13060881144142272,
             -250: 0.08714239823981712,
             -325: 0.048978304290533514,
             -400: 0.023150474092398385,
             -475: 0.009153909907365679,
             -550: 0.002999263753935581,
             -625: 0.0008021305660728393,
             -700: 0.0001711056634994288,
             -775: 2.8080048846470633e-05,
             -850: 3.338786295769374e-06,
             -925: 2.5682971505918264e-07,
             -1000: 9.63111431471935e-09})

In [4]:
result

0.3587510686633638

So we the final probability of making money is 0.3588, a little more than a 1 in 3 chance, which isn't as bad as one might imagine, however definitely more likely you'll lose money.

### Computation via Combinatorics

We can also use combinatorics and probability to compute the odds without running through all of the steps:

#### The Expression

$$ \sum_{A,B,C} \binom{10}{A} \left ( \frac{16}{38} \right ) ^ {A} \binom{10 - A}{B} \left ( \frac{16}{38} \right ) ^ {B} \binom{10 - A - B}{C} \left ( \frac{6}{38} \right ) ^ {C}$$
Such that
$$ 2 A - B - 4 C > 0 $$
$$ A + B + C = 10 $$
$$ A,B,C \geq 0 $$

Where
- $A$ is the number of overlaps, making \\$50
- $B$ is the number of singles, losing \\$25
- $C$ is the number of misses, losing \\$100
- $\binom{x}{y}$ is the binomial coefficient, or "$x \: \mathrm{choose} \: y$"
- $\sum_{A,B,C}$ is the sum over all combinations of $A,B,C$

#### Simplification

We can simplify the equation by subbing in the equality for $C$ and rearranging terms.

$$ \left ( \frac{6}{38} \right ) ^ {10} \sum_{A,B} \binom{10}{A} \binom{10 - A}{B}  \left ( \frac{16}{6} \right ) ^ {A+B} $$
Such that
$$ 6 A + 3B > 40 $$
$$ A+B \leq 10 $$
$$ A,B \geq 0 $$

#### Further Simplification for Symmetry

This can also be rearranged like so: 
$$ 10! \left ( \frac{6}{38} \right ) ^ {10} \sum_{A,B} \frac{1}{A! B! (10-A-B)!} \left ( \frac{16}{6} \right ) ^ {A+B} $$

#### Number of Terms

Now this is almost a closed-form expression for the answer, however one question is how many terms does one actually need to comput in the sum?
An easy upper bound is since both $A$ and $B$ must be between $0$ and $10$ then the number of terms must be less than $100$.
As we'll find out, the actual answer is 16 terms.

In [5]:
import math

In [6]:
result = 0
count = 0
for A in range(10+1):
    for B in range(10-A +1):
        if 6*A + 3*B > 40:
            print("triplet:", A,B,10-A-B)
            count += 1
            result += math.comb(10,A) * math.comb(10-A,B) * (16/6) ** (A+B)
result *= (6/38) ** 10
print("total terms:", count)

triplet: 4 6 0
triplet: 5 4 1
triplet: 5 5 0
triplet: 6 2 2
triplet: 6 3 1
triplet: 6 4 0
triplet: 7 0 3
triplet: 7 1 2
triplet: 7 2 1
triplet: 7 3 0
triplet: 8 0 2
triplet: 8 1 1
triplet: 8 2 0
triplet: 9 0 1
triplet: 9 1 0
triplet: 10 0 0
total terms: 16


In [7]:
result

0.3587510686633636

Again we get the final probability of making money is 0.3588, so that confirms our answer.

## Conclusion

We've seen 2 approaches to solving this problem, namely:

- State Space
- Combinatorics

Along the way we've learned and used techniques from probability, utility theory, and combinatorics. Also we learned how to play roulette!

This was an enjoyable. And we learned no matter what we bet in roullete, the house always has the same edge.

### Further Work

We also could have done a Monte Carlo simulation to get an approximation of our answer.

### Further Reading References and Links

- https://en.wikipedia.org/wiki/Probability
- https://en.wikipedia.org/wiki/Expected_value
- https://en.wikipedia.org/wiki/Utility
- https://en.wikipedia.org/wiki/Expected_utility_hypothesis
- https://en.wikipedia.org/wiki/Combinatorics
- https://en.wikipedia.org/wiki/State_space_(computer_science)
- https://en.wikipedia.org/wiki/Probability_space

### Answers Summary

- The expected result of any roulette bet is a loss of 0.0526 of your bet.
- The probability of making any money using the betting strategy proposed 10 times is 0.3587